# Covid antigen test statistics

Here I will try to calculate the probabilities around antigen tests results and the real probabilities of having Covid19.

In this approach I will use Bayes theorem. 

$$P(H|E) = \frac{P(E|H) * P(H)}{P(E)}$$

Let's see break it down into its components.
- P(H|E) is the probability of a hypothesis being true given an event. In this example, I am interested to know what is the probability of not having Covid given that I got a negative test result.
- P(E|H) is the probability of my test being negative given that I don't have Covid. This is called the "specificity" of the test.
- P(H) is the prior probability of not having Covid.
- P(E) is the probability of a test being negative. This can be broken down even more:

$$P(H|E) = \frac{P(E|H) * P(H)}{P(H)*P(E|H) + P(¬H) * P(E|¬H)}$$

- P(¬H) is the probability of having Covid (remember that my hypothesis H is that I don't have Covid).
- P(E|¬H) is the probability of a test being negative given that I have Covid.

Even though I only need specificity, I will still document specificity and sensitivity in case I need them in the future.

Sensitivity and specificity will come from the most recent studies ([Dinnes et al., 2021](https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD013705.pub2/full); [Turcato et al., 2022](https://www.sciencedirect.com/science/article/pii/S0735675721008573)). 

Both studies used data up to end of 2020, which is not very recent, so I will stick to the most conservative values, considering that antigen tests could be worse at detecting the new Omicron variant of the virus.

Also, I will distinguish between symptomatic and asymptomatic cases, because the statistical properties differ among them.


## Evaluation metrics of antigen tests

The Bayes formula requires sensitivity and specificity and false-positive rate. Sensitivity of a test (true positive rate) is the probability of getting a positive test result given that the person has the condition.

$$Sensitivity = \frac{True Positives}{True Positives + False Negatives}$$

Specificity (false negative rate), on the other hand, refers to the probability of getting a negative result when the person does not have the condition.

$$Specificity = \frac{True Negatives}{True Negatives + False Positives}$$

Below I will include sensitivity and specificity of antigen tests according to different authors. They also include 95% confidence intervals.

### Dinnes et al., 2021

They didn't provide raw data for the tables, although they reported their sensitivity and specificity estimates.

*Symptomatic patients:*

- Sensitivity: 72.0% (63.7–79.0)
- Specificity: 99.5% (98.5–99.8)

*Asymptomatic patients:*

- Sensitivity: 58.1% (40.2–74.1)
- Specificity: 98.9% (93.6–99.8)


### Turcato et al., 2022

*Symptomatic patients:*

|                  | Healthy | Infected |
|------------------|---------|----------|
| Negative antigen | 876     | 30       |
| Positive antigen | 21      | 264      |

- Sensitivity: 89.8% (88.0–91.5)
- Specificity: 97.6% (97.1–98.1)

*Asymptomatic patients*:

|                  | Healthy | Infected |
|------------------|---------|----------|
| Negative antigen | 2594    | 38       |
| Positive antigen | 11      | 65       |

- Sensitivity: 63.1% (58.4–67.8)
- Specificity: 99.6% (99.5–99.7)

In [18]:
# confirming sensitivity and specificity values for asymptomatic patients in Turcato et al., 2022
sensitivity = 65/(65+38)
specificity = 2594/(2594+11)

print('Values for asymptomatic patients (Turcato et al., 2022):')
print("sensitivity:", round(sensitivity, 3))
print("specificity:", round(specificity, 3))

Values for asymptomatic patients (Turcato et al., 2022):
sensitivity: 0.631
specificity: 0.996




### Conclusions, which values to use?

As we said in the beginning, we will use the most conservative values, namely, the ones from Turcato et al. (2022). Moreover, we will just consider the asymptomatic patients, which is my case.

In [23]:
sensitity = 0.581
specificity = 0.989
false_positive_rate = 1 - specificity

## Prior probability of not having Covid

Getting an exact value is not easy, because it has many influencing factors. A good start is the 7-day incidence rate (per 100,000 inhabitants). Luckily this data is [updated very often](https://lua.rlp.de/de/presse/detail/news/News/detail/coronavirus-sars-cov-2-aktuelle-fallzahlen-fuer-rheinland-pfalz/) and it even includes separated statistics for age groups.

    7-day incidence in Kaiserslautern (10.01.2022): 827.

The prior probability of not having Covid should be given by:

$$ 1 - \frac{7-day-incidence}{100,000}$$

In [17]:
prior = 1 - 827/100000

## Plugging in the numbers into Bayes formula

$$P(H|E) = \frac{P(E|H) * P(H)}{P(H)*P(E|H) + P(¬H) * P(E|¬H)}$$


In [24]:
p_E_H = specificity
p_H = prior
P_not_H = 1 - p_H
P_E_not_H = false_positive_rate

In [41]:
p_H_E = p_E_H * p_H / (p_H * p_E_H + P_not_H * P_E_not_H)
print(f'My probability of not being infected given that I have a negative test is {p_H_E*100:.2f}%')

My probability of not being infected given that I have a negative test is 99.99%


That was on Thursday. However, on Friday I had contact with a friend who developed symptoms and had a positive antigen test. She might have infected me during our interaction, so I would like to calculate my probability of having Covid after that event.

We need to update the prior probability accordingly.
